In [1]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  (), please run `%reset?` for details


In [2]:
import pandas as pd
import numpy as np
import pyodbc 

In [3]:
QueryFile = '../00_AuxSQL/01.Vending_Propor.sql'

fd = open(QueryFile,'r', encoding='utf-16')
Query = fd.read()
fd.close()

In [4]:
%%time
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=esmz08srdb009;'
                      'Database=ITE_PRD;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()


Wall time: 867 ms


In [5]:
%%time
Popor_estancos = pd.read_sql(Query,conn)

Wall time: 1min 43s


inputcsv=r'https://spainblob.blob.core.windows.net/altadis/Vending_Proportions.csv'

Popor_estancos = pd.read_csv(inputcsv) 


In [6]:
Popor_estancos.columns

Index(['CUSTOMER_ID', 'Sales2C', 'Sales2C_num', 'MES', 'BLOND_VOL',
       'BLOND_PCK', 'BLACK_VOL', 'BLACK_PCK', 'RYO_VOL', 'RYO_PCK',
       'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK'],
      dtype='object')

In [7]:
Popor_estancos.head()

,CUSTOMER_ID,Sales2C,Sales2C_num,MES,BLOND_VOL,BLOND_PCK,BLACK_VOL,BLACK_PCK,RYO_VOL,RYO_PCK,CIGAR_VOL,PIPE_VOL,PIPE_PCK
0,07008519,None,0.0,201808.0,506172.0,25193.0,0.030029,0.030167,0.043092,0.022665,0.018452,0.000000,0.000000
1,30002141,56% - 60%,58.0,201902.0,527840.0,26102.0,0.021597,0.021684,0.240946,0.111217,0.070750,0.068619,0.027584
2,03020443,6% - 10%,8.0,201906.0,254684.0,12548.0,0.040049,0.040643,0.158050,0.076426,0.016491,0.000000,0.000000
3,08040776,None,0.0,201804.0,106608.0,5262.0,0.084421,0.085518,0.325585,0.157544,0.101352,0.020636,0.011402
4,08060329,16% - 20%,18.0,201804.0,1067616.0,53330.0,0.024540,0.024564,0.139989,0.061110,0.044213,0.000187,0.000187


In [8]:
feats_to_explore=['BLOND_VOL', 'BLOND_PCK', 'BLACK_VOL', 'BLACK_PCK', 'RYO_VOL', 'RYO_PCK', 'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK']
#feats_to_explore=['BLACK_VOL', 'BLACK_PCK', 'RYO_VOL', 'RYO_PCK', 'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK']
#feats_to_explore=[ 'BLOND_PCK',  'BLACK_PCK', 'RYO_PCK', 'CIGAR_VOL', 'PIPE_PCK']
labels= ['0% - 5%', '6% - 10%', '11% - 15%', '16% - 20%', '21% - 25%', '26% - 30%', '31% - 35%', '36% - 40%', '41% - 45%', '46% - 50%', '51% - 55%', '56% - 60%','61% - 65%', '66% - 70%', '71% - 75%', '76% - 80%', '81% - 85%', '86% - 90%', '91% - 95%', '96% - 100%']


In [ ]:
Popor_estancos.shape

(313512, 13)

In [ ]:
from sklearn import preprocessing

lab_enc = preprocessing.LabelEncoder()
Score = lab_enc.fit_transform( Popor_estancos['Sales2C_num'])
Popor_estancos['Score']=Score


In [ ]:
data=Popor_estancos

In [ ]:
data=data.dropna()

In [ ]:
data.shape

(251267, 14)

# Entrenamiento con muestra restrictiva.
Dado que el % de ventas al segundo canal es declarativo, tomare solamente los valores mas "fialbles", esto es, eliminaré outliers de forma muy restritctiva(33%-66%), con estos datos, entrenaré el modelo.

### dataset sin outliers

In [ ]:
#dataset sin outliers

cat_Feat=['CUSTOMER_ID','Sales2C','Sales2C_num','Score','MES']
cat_data = data[cat_Feat]

num_data =data[feats_to_explore]


In [ ]:
from scipy import stats

idx = np.all(stats.zscore(num_data) < 3, axis=1)
Data_cleaned = pd.concat([num_data.loc[idx], cat_data.loc[idx]], axis=1)

In [ ]:
# Restrictive Tiles
Q1 = num_data.quantile(0.33)
Q3 = num_data.quantile(0.66)
IQR = Q3 - Q1
idx = ~((num_data < (Q1 - 1.5 * IQR)) | (num_data > (Q3 + 1.5 * IQR))).any(axis=1)
Data_cleaned = pd.concat([cat_data.loc[idx], num_data.loc[idx]], axis=1)

In [ ]:
print (Data_cleaned.shape)
print('DataSet sin outliers:',round(100*Data_cleaned.shape[0]/ data.shape[0],2),'%')

(133590, 14)
DataSet sin outliers: 53.17 %


In [ ]:
from sklearn.model_selection import train_test_split

Score = lab_enc.fit_transform( Data_cleaned['Sales2C_num'])

X_train, X_test, y_train, y_test = train_test_split(Data_cleaned[feats_to_explore], Score, test_size=0.1, random_state=2019, stratify=Data_cleaned['Sales2C_num'])

# RANDOM FOREST

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

#Create Classifier
n_estimators=400
rfc=RandomForestClassifier(n_estimators=n_estimators,  oob_score = True)
rfc.fit( X_train, y_train);

Wall time: 6min 19s


In [ ]:
print(n_estimators, ': Train Score: {:.2f}%'.format( rfc.score( X_train, y_train)*100))
print(n_estimators, ': out-of-bag Score: {:.2f}%'.format(rfc.oob_score_*100))
print(n_estimators, ': Test Score: {:.2f}%'.format(rfc.score(X_test, y_test)*100))

400 : Train Score: 99.85%
400 : out-of-bag Score: 57.27%
400 : Test Score: 58.28%


In [ ]:
X_train.columns

Index(['BLOND_VOL', 'BLOND_PCK', 'BLACK_VOL', 'BLACK_PCK', 'RYO_VOL',
       'RYO_PCK', 'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK'],
      dtype='object')

# Predicción sobre el conjunto de datos completo

In [ ]:
%%time
predic = rfc.predict(Popor_estancos[feats_to_explore])

In [ ]:
predic.shape, Popor_estancos.shape

In [ ]:
Popor_estancos['predic_ID']=predic
Popor_estancos['predic']=lab_enc.inverse_transform(predic)

In [ ]:
Popor_estancos[Popor_estancos.predic == Popor_estancos.Sales2C_num].count()/Popor_estancos.shape[0]

In [ ]:
Popor_estancos[cat_Feat+['predic']].head(30)

In [ ]:
Popor_estancos [Popor_estancos.CUSTOMER_ID==36002350][feats_to_explore+['predic']]

In [ ]:
Output=r'../02_AuxData/01_Vending_Predic_Prop.csv'

In [ ]:
Popor_estancos[['CUSTOMER_ID', 'MES', 'Sales2C_num', 'predic']].to_csv(Output , sep='|', index=False) 


In [ ]:
#Duración 10 min
    

## Resultado:

la prediccion obtenida en el fichero 01_Vending_Predic_Prop.csv, se guardara en BBDD para la transformación final de los Datos